In [ ]:
# 필요 Library install
!pip install transformers==4.39.2 peft==0.10.0 trl==0.8.6 bitsandbytes==0.43.0 accelerate==0.29.0

In [ ]:
# HF token 설정
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
# Google Drive Import
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
fine_tuned_model_path = "/content/drive/MyDrive/fine_tune_output/checkpoint-50"

In [ ]:
# 모델 경량화: Quantization 설정
from transformers import BitsAndBytesConfig
import torch

quantization_config=BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type='nf4'
)

In [ ]:
# 기본 Llama 3 모델 로드
from transformers import AutoModelForCausalLM
base_model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Meta-Llama-3-8B",
    quantization_config = quantization_config,
    device_map = {"": 0}
)

In [ ]:
# Fine Tune 된 모델 로드
fine_tuned_model_cp_50 = AutoModelForCausalLM.from_pretrained(
    fine_tuned_model_path,
    quantization_config=quantization_config,
    device_map = {"": 0}
)

In [ ]:
# Tokenizer 로드
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(fine_tuned_model_path)

In [ ]:
# Prompt/Response Format 관련 설정
EOS_TOKEN = tokenizer.eos_token

def convert_to_alpaca_format(instruction, response):
    alpaca_format_str = f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.\
    \n\n### Instruction:\n{instruction}\n\n### Response:\n{response}\
    """

    return alpaca_format_str

In [ ]:
def test_model(instruction_str, model):
    inputs = tokenizer(
    [
        convert_to_alpaca_format(instruction_str,"",)
    ], return_tensors = "pt").to("cuda")
    outputs = model.generate(**inputs, max_new_tokens = 128, use_cache = True, temperature = 0.05, top_p = 0.95)
    return(tokenizer.batch_decode(outputs)[0])

In [ ]:
questions = [
    "List three ways to reduce plastic waste in daily life.",
    "Write a haiku about artificial intelligence",
    "How can I improve my public speaking skills?",
    "AI 분야에서 사용하는 LLM이라는 용어가 뭔지 설명해줘",
    "What is a famous tall tower in Paris?",
    "What is Fine-Tuning?",
    "15와 25의 최소공배수를 구하시오.",
    "제2차 세계대전의 주요 원인은 무엇이었나요?",
    "광합성 과정을 간단히 설명해주세요.",
    "셰익스피어의 '햄릿'에서 주인공의 성격을 분석해보세요.",
    "기후 변화가 전 세계적으로 미치는 영향에 대해 설명해주세요.",
    "파이썬에서 리스트와 튜플의 차이점은 무엇인가요?",
    "인공지능은 의식을 가질 수 있을까요? 그 이유는 무엇인가요?",
    "'안녕하세요, 오늘 날씨가 좋네요'를 영어로 번역해주세요.",
    "우주 여행을 주제로 한 짧은 이야기를 만들어보세요.",
    "인플레이션이 경제에 미치는 영향을 설명해주세요."
]

In [ ]:
answers_dict = {
  "base_model_answers": [],
  "fine_tuned_model_answers": []
}
for idx, question in enumerate(questions):
    print(f"Processing EXAMPLE {idx}==============")
    base_model_output = test_model(question, base_model)
    answers_dict['base_model_answers'].append(base_model_output)
    fine_tuned_model_output = test_model(question, fine_tuned_model_cp_50)
    answers_dict['fine_tuned_model_answers'].append(fine_tuned_model_output)

In [ ]:
def simple_format(text, width=120):
    return '\n'.join(line[i:i+width] for line in text.split('\n') for i in range(0, len(line), width))


for idx, question in enumerate(questions):
    print(f"EXAMPLE {idx}==============")
    print(f"Question: {question}")

    print("<<Base Model 답변>>")
    base_model_answer = answers_dict["base_model_answers"][idx].split("### Response:")[1]
    print(simple_format(base_model_answer))
    print("---")
    print("<<Fine Tuning Model 답변>>")
    fine_tuned_model_answer = answers_dict["fine_tuned_model_answers"][idx].split("### Response:")[1]
    print(simple_format(fine_tuned_model_answer))
    print()